In [27]:
import csv

# Sieve of Eratosthenes, used to generate prime numbers sequentially. Used to generate a list of prime numbers
def sieve_of_eratosthenes(limit):
    """Return a list of all prime numbers up to the specified limit using the Sieve of Eratosthenes."""
    primes = []
    is_prime = [True] * (limit + 1)
    is_prime[0] = is_prime[1] = False 
    
    for number in range(2, limit + 1):
        if is_prime[number]:
            primes.append(number)
            for multiple in range(number * number, limit + 1, number):
                is_prime[multiple] = False
    
    return primes


# Used to write primes to .csv in hopes of saving time when brute forcing decryption
def write_primes_to_csv(primes, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Prime Number'])  # Write the header
        for prime in primes:
            writer.writerow([prime])  # Write each prime number in a new row



In [29]:
# Function for generating a valid e and d value for the encryption

def check_e_d(p, q):
    # Temp vars because d may be used throughout this notebook
    d = 0
    coprime = False

    # Go through the list of small primes to check if there is a valid value of d for the given p and q. If not, go to the next prime value
    # for e and see if there is a valid d
    for element in first_primes_list[3:]:
        # We use a try-except here because the pow function will throw a ValueError, so we catch with an 'except' rather than using 'if'
        try: 
            totient = math.lcm(p - 1, q - 1)
            d = pow(element, -1, totient)
            # If d is an integer, we then check if it is coprime to our potential e value. If e is valid, we can return e and d
            if(d != 0 and totient % element != 0):
                return [element, d, totient, True]      
        # If we get an error back, rather than value for d, loop again with the next prime value from the list
        except Exception as ex:
            continue
        # If the whole list of first primes is exhausted, return 0s
        else:
            return [0, 0, 0, False]



In [5]:
# Github Link - https://github.com/skeetercathcart/CS445_FinalProject_F24

import random

# Pre generated primes
first_primes_list = [3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]


# Functions for generating random primes

# Random number generator
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)
 
 
def getLowLevelPrime(n):
    #Generate a prime candidate divisible by first primes
    while True:
        # Obtain a random number
        pc = nBitRandom(n)
 
        # Test divisibility by pre-generated primes
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        else:
            return pc
 
def isMillerRabinPassed(mrc):
    # Run 20 iterations of Rabin Miller Primality test
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Set number of trials here
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True

def generate_primes(bit_size):
    RSA_Primes = []
    while True:
        n = bit_size
        P = getLowLevelPrime(n)
        if not isMillerRabinPassed(P):
            continue
        else:
            break
    while True:
        n = bit_size
        Q = getLowLevelPrime(n)
        if not isMillerRabinPassed(Q):
            continue
        else:
            break
    RSA_Primes.append(P)
    RSA_Primes.append(Q)
    return(RSA_Primes)


In [61]:
import math
from sympy import mod_inverse

# Assign k value
k = 3

# Generate Initial Primes Numbers
coprime = False
P = 0
P = 1
while(coprime == False and P != Q):

    RSA_Primes = generate_primes(8)
    [P, Q] = RSA_Primes

    # Calculate n
    n = P * Q

    # Calculate d (modular multiplicative inverse) and e (small prime) that are coprime
    [e, d, totient, coprime] = check_e_d(P, Q)
    
    

print('Q: ' + str(Q))
print('P: ' + str(P))
print('n: ' + str(n))
print('totient: ' + str(totient))
print('k: ' + str(k))
print('d: ' + str(d))
print('e: ' + str(e))

Q: 223
P: 157
n: 35011
totient: 5772
k: 3
d: 2099
e: 11


In [83]:
# Encrypt a message

message = 'o'

print(message)

# Convert message to their ascii values
encoded_message = message.encode('utf-8')

# Convert bytes to integer (basically a mathematical base conversion)
encoded_int = int.from_bytes(encoded_message, byteorder='big')

# Encrypt the encoded message using the Public Key
public_key = pow(encoded_int, e) % n

print('Public Key: ' + str(public_key))
print('Encoded int: ' + str(encoded_int))


o
Public Key: 11237
Encoded int: 111


In [85]:
# Decrypt the message
private_key = pow(public_key, d) % n

# Convert private key integer into utf-8 byte array
decode_bytes = bytes([private_key])

# Convert byte array back into string
decode_string = decode_bytes.decode('utf-8')

print('Decoded plaintext: ' + str(decode_string))

Decoded plaintext: o


In [93]:
from sympy import isprime, mod_inverse
from itertools import permutations

# Brute force for small primes
def brute_force_rsa(encrypted_int):
    first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19]  # Small primes for demonstration
    
    for p in first_primes_list[5:]:
        for q in first_primes_list[6:]:
            if p != q and isprime(p) and isprime(q):
                
                try:
                    brute_n = p * q
                    [brute_e, brute_d] = check_e_d(p, q)
                    plaintext = pow(encrypted_int, brute_d, brute_n)
                    print(f"Found keys with p={p} and q={q}:")
                    print(f"n = {brute_n}")
                    print(f"Public exponent e = {brute_e}")
                    print(f"Private exponent d = {brute_d}")
                    print(f"Plaintext = {plaintext}")
                    return
                except ValueError:
                    continue

    print("No matching keys found")

# Run the brute force example
brute_force_rsa(encoded_int)

No matching keys found


In [95]:
# Brute force loop
def brute_force_rsa(encrypted_int):
    
    for brute_p in first_primes_list:
        for brute_q in first_primes_list:
            if brute_p != brute_q and isprime(brute_p) and isprime(brute_q):
                brute_n = brute_p * brute_q
                try:
                    [brute_e, brute_d] = check_e_d(brute_p, brute_q)
                    if (pow(encrypted_int, brute_d) % n):
                        print(f"Found keys with p={brute_p} and q={brute_q}:")
                        print(f"n = {brute_n}")
                        print(f"Public exponent e = {brute_e}")
                        print(f"Private exponent d = {brute_d}")
                        print(f"Plaintext = kek")
                        return
                except ValueError:
                    continue

    print("No matching keys found")

# Run the brute force example
brute_force_rsa(encoded_int)

No matching keys found
